Organize, filter, and clean excel file 
- original file has multiple sheets (for different data set types (e.g. single visit, imaging, labs etc)
- only interested in deployed controls ('C') and mTBI ('T')
- each participant can have multiple visits for each visit sequence (1-3 visit sequences per participant)

In [ ]:
#getting and working with data
import pandas as pd
import numpy as np
import re
import os
import datetime as dt
import string

#visualizing results
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
sns.set_context("poster")
sns.set_style("ticks")
#import yellowbrick as yb

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 15000)
pd.set_option('display.max_colwidth', -1)

import warnings; warnings.simplefilter('ignore')
np.set_printoptions(suppress=True)

## explore file contents and visit meta data

In [ ]:
path_data = 'C:/Users/Schindler/Documents/Schindler_Lab/Data/Clinical projects/PeskindTBI/freeze_200219/TBIFreeze_20200428_python.xlsx'

whole_file = pd.ExcelFile(path_data)
#get list of sheet names in file
print(whole_file.sheet_names, '\n')

In [ ]:
#original file has multiple sheets (for different data set types (e.g. single visit, imaging, labs etc)
#only interested in deployed controls ('C') and mTBI ('T')
#each participant can have multiple visits for each visit sequence (1-3 visit sequences per participant)
#first want to explore pattern of visits and determine which sheets have multiple visits per visit sequence

for sheet in whole_file.sheet_names:
    
    sheet_data_int = pd.DataFrame()
    
    print('Sheet being processed:\n', sheet)
    
    #create intermediate dataframe
    data_int = pd.DataFrame(data = pd.read_excel(whole_file, sheet))

    #select only TBIID C and T (control and TBI)
    data_int = data_int[data_int['TBIID'].str.match(r'[CT]\d\d')]
    print('Data shape only deployed controls and mTBI groups:\n', data_int.shape)

    #meta data
    visit_sequences = data_int['VisitSeq'].unique()
    print('Number of visit sequences:\n', len(visit_sequences))
    print('Visit seq breakdown:\n', visit_sequences)
    
    #meta data
    participants_total = data_int['TBIID'].unique()
    print('Number of participants:\n', len(participants_total), '\n')
    
    #for each visit
    for visit in visit_sequences:
        #determine max number of visits for this visit sequence
        data_int_visit = data_int[data_int['VisitSeq'] == visit]
        max_visits = data_int_visit.groupby('TBIID')['TBIID'].count().max()
        print(f'In visits sequence {visit} there is/are a max of {max_visits} visit(s) \n')
        
    print('\n', '\n')

## combine sheets and use mean for sheets with multiple visits per sequence

In [ ]:
#some sheets have multiple visits per visit sequence and some sheets have a single visit for each visit sequence
#use mean across visits for sheets that have data from multiple visits within a visit sequence

data_visit_mean = pd.DataFrame()

for sheet in whole_file.sheet_names:
    
    sheet_data = pd.DataFrame()
    
    print('Sheet being processed:\n', sheet)
    
    #create intermediate dataframe
    data_sheet = pd.DataFrame(data = pd.read_excel(whole_file, sheet))

    #select only TBIID C and T (control and TBI)
    data_sheet = data_sheet[data_sheet['TBIID'].str.match(r'[CT]\d\d')]
    print('Data shape only deployed controls and mTBI groups:\n', data_sheet.shape)

    #meta data
    visit_sequences = data_sheet['VisitSeq'].unique()
    print('Number of visit sequences:\n', len(visit_sequences))
    
    #meta data
    participants_total = data_sheet['TBIID'].unique()
    print('Number of participants:\n', len(participants_total), '\n')
    
    #clean
    data_sheet = data_sheet.replace({-999.0: np.nan, 999: np.nan, 'None': np.nan})
    
    #for each visit
    for visit in visit_sequences:
        print('Visit sequence being processed:\n', visit)
        
        #get visit data
        data_sheet_visit = data_sheet[data_sheet['VisitSeq'] == visit]

        #determine max number of visits for this visit sequence
        max_visits = data_sheet_visit.groupby('TBIID')['TBIID'].count().max()

        #make new df to fill with cleaned data
        data_sheet_visit_merge = pd.DataFrame()

        #if only one visit per TBIID in visit sequence, do nothing
        if max_visits == 1:
            data_sheet_visit_merge = data_sheet_visit_merge.append(data_sheet_visit, ignore_index=True, sort=False)
        #if more than one visit per TBIID in visit sequence then need to get mean across visits
        else:
            for TBIID in data_sheet_visit['TBIID'].unique():
                data_sheet_visit_TBIID = data_sheet_visit[data_sheet_visit['TBIID'] == TBIID]
                data_sheet_visit_TBIID_mean = data_sheet_visit_TBIID.mean()
                data_sheet_visit_TBIID_mean['TBIID'] = TBIID
                data_sheet_visit_TBIID_mean['VisitSeq'] = visit
                data_sheet_visit_merge = data_sheet_visit_merge.append(data_sheet_visit_TBIID_mean, ignore_index=True, sort=False)
        
        #combine visit seq with other seq from same sheet
        print('Data shape for current visit sequence:\n', data_sheet_visit_merge.shape, '\n')
        sheet_data = sheet_data.append(data_sheet_visit_merge, ignore_index=True, sort=False)
        
    #combine sheet with other sheets 
    print('Data shape for current sheet:\n', sheet_data.shape, '\n')
    if data_visit_mean.shape[0] < 1:
        data_visit_mean = sheet_data
    else:
        data_visit_mean = pd.merge(data_visit_mean, sheet_data, how='left', on=['TBIID', 'VisitSeq'], sort=False)
        
    print('Data shape for current final dataframe:\n', data_visit_mean.shape, '\n')

#add group column 
data_visit_mean['Group'] = [TBIID[0] for TBIID in data_visit_mean['TBIID']]

print('Final shape of data_visit_mean:\n', data_visit_mean.shape)
data_visit_mean.head()

## combine sheets and use mean for sheets with multiple visits per sequence

In [ ]:
#some sheets have multiple visits per visit sequence and some sheets have a single visit for each visit sequence
#we are currently interested in data only from the first visit of each visit sequence

first_visits = pd.DataFrame()

#each data set uses as different column name for visit date, use a dictionary to access corresponding date column name
sheet_dic = {'single': 'ScreenDate', 
             'mult': 'MeasureDate',
             'TBI': 'NSIFormDate',
             'Qfup': 'ScreenDate',   
             'DTI': 'ScreenDate', 
             'PET': 'ScreenDate',
             'labs': 'CLabDate'}

for sheet in whole_file.sheet_names:
    
    sheet_data_int = pd.DataFrame()
    
    print('Sheet being processed:\n', sheet)
    
    #create intermediate dataframe
    data_int = pd.DataFrame(data = pd.read_excel(whole_file, sheet))

    #select only TBIID C and T (control and TBI)
    data_int = data_int[data_int['TBIID'].str.match(r'[CT]\d\d')]
    print('Data shape only deployed controls and mTBI groups:\n', data_int.shape)

    #meta data
    visit_sequences = data_int['VisitSeq'].unique()
    print('Number of visit sequences:\n', len(visit_sequences))
    
    #meta data
    participants_total = data_int['TBIID'].unique()
    print('Number of participants:\n', len(participants_total), '\n')
    
    #for each visit
    for visit in visit_sequences:
        print('Visit sequence being processed:\n', visit)
        
        #get visit data
        full_visit_seq_data = data_int[data_int['VisitSeq'] == visit]
        visit_data_int = pd.DataFrame()
        
        #loop through participants and for each one find and save the first visit of that visit sequence
        participants_visit = full_visit_seq_data['TBIID'].unique()
        #print('Number of participants for this visit sequence:\n', len(participants_visit))
        for part in participants_visit:
            dates = full_visit_seq_data.loc[(full_visit_seq_data['TBIID'] == part), sheet_dic[sheet]].values
            min_date = dates.min()
    
            visit_seq_data_indiv = full_visit_seq_data[(full_visit_seq_data['TBIID'] == part) & (full_visit_seq_data[sheet_dic[sheet]] == min_date)]
            visit_data_int = visit_data_int.append(visit_seq_data_indiv)

        #reset indexes and clean up missing values
        visit_data_int = visit_data_int.reset_index(drop=True)
        visit_data_int = visit_data_int.replace({-999.0: np.nan, 999: np.nan, 'None': np.nan})
        
        if sheet_data_int.shape[0] < 1:
            sheet_data_int = visit_data_int
        else:
            sheet_data_int = sheet_data_int.append(visit_data_int)
 
    #combine into one final df that contains all first visit for each sequence
    if first_visits.shape[0] < 1:
        first_visits = sheet_data_int
    else:
        first_visits = pd.merge(first_visits, sheet_data_int, how='left', on=['TBIID', 'VisitSeq'], sort=False)
    print('Visit data shape for current sheet:\n', first_visits.shape, '\n')

#clean up duplicates
first_visits.drop_duplicates(inplace=True)

#add group column 
first_visits['Group'] = [TBIID[0] for TBIID in first_visits['TBIID']]

print('Final shape of first visit data:\n', first_visits.shape)
first_visits.head()

In [ ]:
data_final = data_visit_mean

## add in RH data

In [ ]:
#read in data from RH pull and combine with df
path_RH = 'C:/Users/Schindler/Documents/Schindler_Lab/Data/Clinical projects/PeskindTBI/RHdatapull/RHpull.csv'

data_RH = pd.read_csv(path_RH)
data_RH = pd.DataFrame(data = data_RH)
print('Data shape all groups:\n', data_RH.shape)

#select only TBIID C and T (control and TBI)
data_RH = data_RH[data_RH['TBIID'].str.match(r'[CT]\d\d')]
print('Data shape only deployed controls and mTBI groups:\n', data_RH.shape)

#clean up missing values
data_RH = data_RH.replace({-999.0: np.nan, 'None': np.nan})

#change DA = 0 to np.nan, add ratio cals
data_RH['DA'] = data_RH['DA'].replace({0: np.nan})
data_RH['da_dopa_ratio'] = data_RH['DA'] / data_RH['DOPA']
data_RH['dopac_da_ratio'] = data_RH['DOPAC'] / data_RH['DA']
data_RH['ne_dopa_ratio'] = data_RH['NE'] / data_RH['DOPA']
data_RH['dhpg_ne_ratio'] = data_RH['DHPG'] / data_RH['NE']

#add to df containing first visit info
data_final = pd.merge(data_final, data_RH, how='left', on=['TBIID', 'VisitSeq'], sort=False)

print('Final shape of first visit data:\n', data_final.shape)
data_final.head()

## add in MESO data

In [ ]:
#read in data from MSD multiplex on blood and CSF and combine with df
path_MESO = 'C:/Users/Schindler/Documents/Schindler_Lab/Data/Clinical projects/PeskindTBI/MSD_for_Abbie.xlsx'

data_MESO = pd.read_excel(path_MESO)
data_MESO = pd.DataFrame(data = data_MESO)
print('Data shape all groups:\n', data_MESO.shape)

#drop participants that have no data
data_MESO.dropna(axis=0, thresh=3, inplace=True)
print('Data shape all groups:\n', data_MESO.shape)
#drop analytes that have many missing values (suggests sensitivity of MSD not sufficient for this analyte)
data_MESO.dropna(axis=1, thresh=100, inplace=True)
print('Data shape all groups:\n', data_MESO.shape)

#add to df containing first visit info
data_final = pd.merge(data_final, data_MESO, how='left', on=['TBIID', 'VisitSeq'], sort=False)

print('Final shape of first visit data:\n', data_final.shape)
data_final.head()

## compute subscales

In [ ]:
#create new column for sum of distance from blast (worst 5)
data_final['QEDist_sum'] = data_final.loc[:, 'QEDist1':'QEDist5'].sum(axis=1)
#create new column for mean of distance from blast (worst 5)
data_final['QEDist_mean'] = data_final.loc[:, 'QEDist1':'QEDist5'].mean(axis=1)
#create new column for min of distance from blast (worst 5)
data_final['QEDist_min'] = data_final.loc[:, 'QEDist1':'QEDist5'].min(axis=1)

#create new columns for NSI 4-factor scoring approach
NSI_comp_vestibular = ['tbiDizzy', 'tbiBalan', 'tbiCoord']
NSI_comp_somatosensory = ['tbiHeada', 'tbiNaus', 'tbiVision', 'tbiLight', 'tbiNoise', 'tbiTingl', 'tbiTstsml']
NSI_comp_cognitive = ['tbiConc', 'tbiForget', 'tbiDecis', 'tbiSlow']
NSI_comp_affective = ['tbiEnergy', 'tbiSleep', 'tbiAnx', 'tbiSad', 'tbiIrrit', 'tbiOverw']
NSI_comp_ERP_affective = ['tbiDisin', 'tbiWithd', 'tbiMoods', 'tbiFight']
NIS_comp_ERP_vestsom = ['tbiRing', 'tbiSpch']

data_final['NSI_vestibular'] = data_final.loc[:, NSI_comp_vestibular].mean(axis=1)
data_final['NSI_somatosensory'] = data_final.loc[:, NSI_comp_somatosensory].mean(axis=1)
data_final['NSI_cognitive'] = data_final.loc[:, NSI_comp_cognitive].mean(axis=1)
data_final['NSI_affective'] = data_final.loc[:, NSI_comp_affective].mean(axis=1)
data_final['NSI_ERP_affective'] = data_final.loc[:, NSI_comp_ERP_affective].mean(axis=1)
data_final['NIS_ERP_vestsom'] = data_final.loc[:, NIS_comp_ERP_vestsom].mean(axis=1)

#create new columns for PCL subscores for 4 factor model from King et al., 1998
PCL_reexp = ['PCL1', 'PCL2', 'PCL3', 'PCL4', 'PCL5']
PCL_avoid = ['PCL6', 'PCL7']
PCL_numb = ['PCL8', 'PCL9', 'PCL10', 'PCL11', 'PCL12']
PCL_hyper = ['PCL13', 'PCL14', 'PCL15', 'PCL16', 'PCL17']

data_final['PCL_reexp'] = data_final.loc[:, PCL_reexp].mean(axis=1)
data_final['PCL_avoid'] = data_final.loc[:, PCL_avoid].mean(axis=1)
data_final['PCL_numb'] = data_final.loc[:, PCL_numb].mean(axis=1)
data_final['PCL_hyper'] = data_final.loc[:, PCL_hyper].mean(axis=1)

#create new columns for PHQ-9 subscores for 2 factor model
PHQ_psych = ['PHQ1', 'PHQ2', 'PHQ6', 'PHQ9']
PHQ_somatic = [ 'PHQ3', 'PHQ4', 'PHQ5', 'PHQ7', 'PHQ8'] 

data_final['PHQ_psych'] = data_final.loc[:, PHQ_psych].mean(axis=1)
data_final['PHQ_somatic'] = data_final.loc[:, PHQ_somatic].mean(axis=1)
        
print(data_final.shape)
data_final.head(1)

## compute health metrics

In [ ]:
#create bins
BMI_bin = []
for value in data_final['BMI'].values:
    if value <= 20:
        BMI_bin.append(20)
    elif 20 <= value <= 25:
        BMI_bin.append(25)
    elif 25 <= value <= 30:
        BMI_bin.append(30)
    elif 30 <= value <= 35:
        BMI_bin.append(35)
    elif value >= 35:
        BMI_bin.append(40)
    else: 
        BMI_bin.append(np.nan)

print(len(BMI_bin))
data_final['BMI_bin'] = BMI_bin
print(data_final.shape)

In [ ]:
#cholesterol related measures 
data_final['Total_HDL_ratio'] = data_final['TotalChol'] / data_final['HDL']
data_final['HDL_LDL_ratio'] = data_final['HDL'] / data_final['LDL']
data_final['LDL_HDL_ratio'] = data_final['LDL'] / data_final['HDL']
data_final['Tri_HDL_ratio'] = data_final['Trig'] / data_final['HDL']

## save to csv

In [ ]:
data_final.to_csv('data_final.csv', index=False)